In [2]:
import tensorflow as tf
import os, random, math, sys
import numpy as np
from PIL import Image

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 验证集数量
_NUM_TEST = 500

# 随机种子
_RANDOM_SEED = 0

# 数据集路径
DATASET_DIR = "E:/Python/tensorflow/captcha/image/"

# tfrecord路径
TFRECORD_DIR = "E:/Python/tensorflow/captcha/"

# 判断tf文件是否存在
def _dataset_exists(dataSet_dir):
    for split_name in ['train','test']:
        output_filename = os.path.join(dataSet_dir,split_name + '.tfrecords')
        if not tf.gfile.Exists(output_filename):
            return False
    return True

# 获取所有验证码图片
def _get_filenames_and_classes(dataSet_dir):
    photo_filenames = []
    for filename in os.listdir(dataSet_dir):
        path = os.path.join(dataSet_dir,filename)
        photo_filenames.append(path)
    return photo_filenames
    
def int64_feature(values):
    if not isinstance(values,(tuple,list)):
        values = [values]
        return tf.train.Feature(int64_list=tf.train.Int64List(value=values))
    
def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def image_to_tfexample(image_data,label0,label1,label2,label3):
    return tf.train.Example(features=tf.train.Features(feature={
        'image':bytes_feature(image_data),
        'label0':int64_feature(label0),
        'label1':int64_feature(label1),
        'label2':int64_feature(label2),
        'label3':int64_feature(label3),
    }))
        
# 把数据转换为tfrecord格式
def _convert_dataset(split_name,filenames,dataSet_dir):
    assert split_name in ['train','test']
    
    with tf.Session() as sess:
        output_filename = os.path.join(TFRECORD_DIR,split_name+'.tfrecords')
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writers:
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d/%d' % (i+1,len(filenames)))
                    sys.stdout.flush()
                    
                    # 读取图片
                    image_data = Image.open(filename)
                    # 根据模型结构resize
                    image_data = image_data.resize((224,224))
                    # 灰度化
                    image_data = np.array(image_data.convert('L'))
                    # 图片转为bytes
                    image_data = image_data.tobytes()
                    
                    # 获取label
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        num_labels.append(int(labels[j]))
                        
                    # 生成protocol数据类型
                    example = image_to_tfexample(image_data,num_labels[0],num_labels[1],num_labels[2],num_labels[3])
                    tfrecord_writers.write(example.SerializeToString())
                    
                except IOError as e:
                    print('Could not read:',filename)
                    print('Error:',e)
                    print('Skip it \n')
    sys.stdout.write('\n')
    sys.stdout.flush()
    
# 判断tfrecord是否存在
if _dataset_exists(TFRECORD_DIR):
    print('tfrecord文件已存在')
    
else:
#     获取所有图片
    photo_filenames = _get_filenames_and_classes(DATASET_DIR)
    photo_filenames
    random.seed(_RANDOM_SEED)
    random.shuffle(photo_filenames)
    train_filenames = photo_filenames[_NUM_TEST:]
    testing_filenames = photo_filenames[:_NUM_TEST]
    
    _convert_dataset('train',train_filenames,DATASET_DIR)
    _convert_dataset('test',testing_filenames,DATASET_DIR)
    print('生成tfrecord文件')
                    
        

>> Converting image 535/535
>> Converting image 500/500
生成tfrecord文件
